<a href="https://colab.research.google.com/github/KimaniKibuthu/brain-tumor-classification/blob/main/Brain_Tumor_Radiogenomic_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RSNA-MICCAI Brain Tumor Radiogenomic Classification



# Introduction

A malignant tumor in the brain is a life-threatening condition. Known as glioblastoma, it's both the most common form of brain cancer in adults and the one with the worst prognosis, with median survival being less than a year. The presence of a specific genetic sequence in the tumor known as MGMT promoter methylation has been shown to be a favorable prognostic factor and a strong predictor of responsiveness to chemotherapy.



Currently, genetic analysis of cancer requires surgery to extract a tissue sample. Then it can take several weeks to determine the genetic characterization of the tumor. Depending upon the results and type of initial therapy chosen, a subsequent surgery may be necessary. If an accurate method to predict the genetics of the cancer through imaging (i.e., radiogenomics) alone could be developed, this would potentially minimize the number of surgeries and refine the type of therapy required.

The Radiological Society of North America (RSNA) has teamed up with the Medical Image Computing and Computer Assisted Intervention Society (the MICCAI Society) to improve diagnosis and treatment planning for patients with glioblastoma. In this competition you will predict the genetic subtype of glioblastoma using MRI (magnetic resonance imaging) scans to train and test your model to detect for the presence of MGMT promoter methylation.

If successful, you'll help brain cancer patients receive less invasive diagnoses and treatments. The introduction of new and customized treatment strategies before surgery has the potential to improve the management, survival, and prospects of patients with brain cancer.

# Evaluation Metric
The Evaluation metric to be used is area under the ROC curve.

# Libraries and Functions

In [ ]:
!pip uninstall kaggle
!pip install --upgrade kaggle

Uninstalling kaggle-1.5.12:
  Would remove:
    /usr/local/bin/kaggle
    /usr/local/lib/python3.7/dist-packages/kaggle-1.5.12.dist-info/*
    /usr/local/lib/python3.7/dist-packages/kaggle/*
Proceed (y/n)? y
  Successfully uninstalled kaggle-1.5.12
     |████████████████████████████████| 61kB 3.3MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=ff972ee74726b36660b42fec74a8514e85b3d0c57241da36f4e6b89cc32eb884
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle


In [ ]:
# import the necessary libraries

# General libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Modelling libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, GlobalAveragePooling2D
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input

In [ ]:
# Variables

TARGET_SIZE = 224
TEST_SIZE = 0.5
BATCH_SIZE = 64
RANDOM_STATE = 42
EPOCHS = 10
LR = 0.0001
WEIGHTS = {
    0:1.95,
    1:0.67
}

# Load Data

In [ ]:
# Load credentials
def credentials(name, token_id):
   # Setup the username and ID
  os.environ["KAGGLE_USERNAME"] = name
  os.environ["KAGGLE_KEY"] = token_id

  print('Done!')

In [ ]:
# Download data
credentials('kimanikibuthu', 'f85c0bb5d43058fddcce7902e1325677')

!kaggle competitions download -c rsna-miccai-brain-tumor-radiogenomic-classification

Done!
100% 12.3G/12.3G [05:00<00:00, 100MB/s] 
100% 12.3G/12.3G [05:00<00:00, 44.0MB/s]


In [ ]:
# Unzip the data
!unzip /content/rsna-miccai-brain-tumor-radiogenomic-classification.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/00445/T1wCE/Image-301.dcm  
  inflating: train/00445/T1wCE/Image-302.dcm  
  inflating: train/00445/T1wCE/Image-303.dcm  
  inflating: train/00445/T1wCE/Image-304.dcm  
  inflating: train/00445/T1wCE/Image-305.dcm  
  inflating: train/00445/T1wCE/Image-306.dcm  
  inflating: train/00445/T1wCE/Image-307.dcm  
  inflating: train/00445/T1wCE/Image-308.dcm  
  inflating: train/00445/T1wCE/Image-309.dcm  
  inflating: train/00445/T1wCE/Image-31.dcm  
  inflating: train/00445/T1wCE/Image-310.dcm  
  inflating: train/00445/T1wCE/Image-311.dcm  
  inflating: train/00445/T1wCE/Image-312.dcm  
  inflating: train/00445/T1wCE/Image-313.dcm  
  inflating: train/00445/T1wCE/Image-314.dcm  
  inflating: train/00445/T1wCE/Image-315.dcm  
  inflating: train/00445/T1wCE/Image-316.dcm  
  inflating: train/00445/T1wCE/Image-32.dcm  
  inflating: train/00445/T1wCE/Image-33.dcm  
  inflating: train/00445/T1wCE/Image-34.dcm  
  inflating: 

In [ ]:
# View the data
def show_images(base_path, class_name):
    # Get images 
    print(f'{class_name}')
    path = os.path.join(base_path, class_name)
    images = os.listdir(path)
    # Extract 16 random images from it
    random_images = [np.random.choice(images) for i in range(8)]

    # Adjust the size of your images
    plt.figure(figsize=(16,12))

    # Iterate and plot random images
  
    for i in range(8):
        plt.subplot(2,4,i+1)
        img = plt.imread(os.path.join(path, random_images[i]))
        
        try:
          plt.imshow(img, cmap='gray')
          #plt.axis('off')
        except FileNotFoundError:
          pass

    # Adjust subplot parameters to give specified padding
    plt.tight_layout()